# Creating Agents with Custom Orchestration

In this folder we will show you how to create an agent using a custom orchestration strategy. 

[Amazon Bedrock Agents](https://aws.amazon.com/bedrock/agents/) streamlines the development of generative AI applications by offering a fully managed solution that uses foundation models and augmenting tools to autonomously run tasks and achieve objectives through orchestrated, multi-step workflows. The default orchestration strategy, Reasoning and Action ([ReAct](https://arxiv.org/abs/2210.03629)), enables users to quickly build and deploy agentic solutions. ReAct is a general problem-solving approach that leverages the foundation model's planning capabilities to dynamically adjust actions at each step. While ReAct offers flexibility by allowing agents to continuously re-evaluate their decisions based on shifting requirements, its iterative approach can lead to higher latency when a large number of tools are involved. 

For greater orchestration control, Amazon Bedrock Agents has launched the [custom orchestrator]() feature, which enables users to fine-tune agent behavior and manage tool interactions at each workflow step. This customization allows organizations to tailor agent functionality to their specific operational needs, improving precision, adaptability, and efficiency. 

In this notebook, we’ll reuse our restaurant example to explore how custom orchestrators work and demonstrate their application with an Reasoning Without Observation ([ReWOO](https://arxiv.org/abs/2305.18323)) example. The image below shows the architecture of the agent we will create:

<center>
  <img src="images/architecture.png" width="65%" />
</center>

During this notebook we will:
1. Create an [Amazon Knowledge Base](https://aws.amazon.com/bedrock/knowledge-bases/) to index our restaurant menus
2. Create and test our restaurant assistant using the default ReAct strategy
3. Create and test our restaurant assistant using the new custom orchestration feature with ReWoo
4. Delete all components to avoid unexpected costs

## Installing and importing prerequisites
Before starting let's update our boto3 packages with the latest functionalities and install any pre-requisite packages

In [1]:
!python3 -m pip install --force-reinstall --no-cache -r requirements.txt 

Looking in indexes: https://pypi.org/simple, https://plugin.us-east-1.prod.workshops.aws
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 38.9 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 59.7 MB/s eta 0:00:00
  Attempting uninstall: Events
    Found existing installation: Events 0.5
    Uninstalling Events-0.5:
      Successfully uninstalled Events-0.5
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0
  Attempting uninstall: six
    Found existing installation: six 1.17.0
    Uninstalling six-1.17.0:
      Successfully uninstalled six-1.17.0
  Attempting uninstall: retrying
    Found existing installation: retrying 1.4.1
    Uninstalling retrying-1.4.1:
      Successfully uninstalled retrying-1.4.1
  Attempting uninstall: PyYAML
    Found existing installatio

Now we can import the required packages for this example.

We will also inport some helper functionalities available in `agents.py` and `knowledge_bases.py`. Those functions will help us creating the knowledge base and our agents easier. They use the boto3 clientes for `bedrock-agents` and `bedrock-agents-runtime`. You can check the implementation those functions in the provided files. In this notebook we will highlight the differences in the [CreateAgent](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent_CreateAgent.html) and [InvokeAgent](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_agent-runtime_InvokeAgent.html) APIs for the custom orchestrator.

In [2]:
import boto3
import json
import time
import sys
import os
from agents import create_agent, invoke_agent_helper, clean_up_resources
from knowledge_bases import KnowledgeBasesForAmazonBedrock

Boto3 version: 1.39.11


## 1. Creating Knowledge Base
Let's create a restaurant assistant Knowledge Base to index the menus of our restaurant. To do so we first need to set some constants for the knowledge base name and description as well as a name for the s3 bucket that will store the documents indexed in the knowledge base

In [3]:
s3_client = boto3.client('s3', region_name="us-west-2")
sts_client = boto3.client('sts', region_name="us-west-2")
session = boto3.session.Session(region_name="us-west-2")
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
knowledge_base_name = f'restaurant-kb'
suffix = f"{region}-{account_id}"
knowledge_base_description = "Knowledge Base containing the restaurant menu's collection"
bucket_name = f'restaurant-kb-{suffix}'

In [4]:
kb = KnowledgeBasesForAmazonBedrock()
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)


[2025-07-23 02:51:56,281] p95550 {credentials.py:1355} INFO - Found credentials in shared credentials file: ~/.aws/credentials
[2025-07-23 02:51:57,060] p95550 {knowledge_bases.py:289} INFO - Checking if S3 bucket 'restaurant-kb-us-west-2-071040227595' exists...


Creating KB restaurant-kb
Step 1 - Creating or retrieving restaurant-kb-us-west-2-071040227595 S3 bucket for Knowledge Base documents


[2025-07-23 02:51:57,337] p95550 {knowledge_bases.py:292} INFO - ✓ S3 bucket restaurant-kb-us-west-2-071040227595 already exists - retrieving it!
[2025-07-23 02:51:57,338] p95550 {knowledge_bases.py:413} INFO - Creating IAM policies for Knowledge Base execution role...
[2025-07-23 02:51:57,339] p95550 {knowledge_bases.py:417} INFO - Creating foundation model policy: AmazonBedrockFoundationModelPolicyForKnowledgeBase_673


Bucket restaurant-kb-us-west-2-071040227595 already exists - retrieving it!
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_673) and Policies


[2025-07-23 02:51:57,572] p95550 {knowledge_bases.py:425} INFO - Foundation model policy AmazonBedrockFoundationModelPolicyForKnowledgeBase_673 already exists, retrieving it
[2025-07-23 02:51:57,659] p95550 {knowledge_bases.py:431} INFO - ✓ Retrieved existing foundation model policy: AmazonBedrockFoundationModelPolicyForKnowledgeBase_673
[2025-07-23 02:51:57,659] p95550 {knowledge_bases.py:444} INFO - Creating S3 access policy: AmazonBedrockS3PolicyForKnowledgeBase_673
[2025-07-23 02:51:57,725] p95550 {knowledge_bases.py:451} INFO - S3 access policy AmazonBedrockS3PolicyForKnowledgeBase_673 already exists, retrieving it


...

Step 6 - Creating Knowledge Base
Checking if index exists: https://r2uwlb0wnin8rn5l2inl.us-west-2.aoss.amazonaws.com/restaurant-kb-index-673


[2025-07-23 02:59:32,942] p95550 {knowledge_bases.py:1220} INFO - Creating knowledge base: restaurant-kb
[2025-07-23 02:59:32,943] p95550 {knowledge_bases.py:1221} INFO - Configuration - Embedding model: amazon.titan-embed-text-v2:0, Collection ARN: arn:aws:aoss:us-west-2:071040227595:collection/r2uwlb0wnin8rn5l2inl
[2025-07-23 02:59:32,943] p95550 {knowledge_bases.py:1224} INFO - Attempting to create knowledge base...


Index restaurant-kb-index-673 already exists
{'type': 'VECTOR', 'vectorKnowledgeBaseConfiguration': {'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}}


[2025-07-23 02:59:33,736] p95550 {knowledge_bases.py:1241} INFO - ✓ Successfully created knowledge base with ID: MIV8LAUJT2
[2025-07-23 02:59:33,738] p95550 {knowledge_bases.py:1284} INFO - Creating data source for knowledge base: MIV8LAUJT2
[2025-07-23 02:59:33,739] p95550 {knowledge_bases.py:1286} INFO - Attempting to create data source...
[2025-07-23 02:59:33,896] p95550 {knowledge_bases.py:1301} INFO - ✓ Successfully created data source with ID: MMKMP973CF


{ 'createdAt': datetime.datetime(2025, 7, 23, 7, 59, 33, 269776, tzinfo=tzutc()),
  'description': "Knowledge Base containing the restaurant menu's collection",
  'knowledgeBaseArn': 'arn:aws:bedrock:us-west-2:071040227595:knowledge-base/MIV8LAUJT2',
  'knowledgeBaseConfiguration': { 'type': 'VECTOR',
                                  'vectorKnowledgeBaseConfiguration': { 'embeddingModelArn': 'arn:aws:bedrock:us-west-2::foundation-model/amazon.titan-embed-text-v2:0'}},
  'knowledgeBaseId': 'MIV8LAUJT2',
  'name': 'restaurant-kb',
  'roleArn': 'arn:aws:iam::071040227595:role/AmazonBedrockExecutionRoleForKnowledgeBase_673',
  'status': 'CREATING',
  'storageConfiguration': { 'opensearchServerlessConfiguration': { 'collectionArn': 'arn:aws:aoss:us-west-2:071040227595:collection/r2uwlb0wnin8rn5l2inl',
                                                                   'fieldMapping': { 'metadataField': 'text-metadata',
                                                                        

### Syncronizing data to knowledge base

Next let's syncronize the knowledge base to index the menus

In [5]:
def upload_directory(path, bucket_name):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,file)

In [6]:
upload_directory("kb_docs", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

uploading file kb_docs/Restaurant_Childrens_Menu.pdf to restaurant-kb-us-west-2-071040227595
uploading file kb_docs/Restaurant_Dinner_Menu.pdf to restaurant-kb-us-west-2-071040227595
uploading file kb_docs/Restaurant_week_specials.pdf to restaurant-kb-us-west-2-071040227595


[2025-07-23 03:00:34,865] p95550 {knowledge_bases.py:1350} INFO - Starting data synchronization for knowledge base: MIV8LAUJT2, data source: MMKMP973CF
[2025-07-23 03:00:34,866] p95550 {knowledge_bases.py:1353} INFO - Waiting for knowledge base to be available...
[2025-07-23 03:00:34,995] p95550 {knowledge_bases.py:1371} INFO - ✓ Knowledge base status: ACTIVE
[2025-07-23 03:00:34,995] p95550 {knowledge_bases.py:1391} INFO - Starting ingestion job...
[2025-07-23 03:00:35,938] p95550 {knowledge_bases.py:1398} INFO - ✓ Ingestion job started with ID: YZKQMMB3SV
[2025-07-23 03:00:35,940] p95550 {knowledge_bases.py:1413} INFO - Monitoring ingestion job progress...
[2025-07-23 03:00:35,940] p95550 {knowledge_bases.py:1424} INFO - Ingestion job status: STARTING (check #1/120)


{ 'dataSourceId': 'MMKMP973CF',
  'ingestionJobId': 'YZKQMMB3SV',
  'knowledgeBaseId': 'MIV8LAUJT2',
  'startedAt': datetime.datetime(2025, 7, 23, 8, 0, 35, 932542, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 7, 23, 8, 0, 35, 932542, tzinfo=tzutc())}


[2025-07-23 03:00:46,074] p95550 {knowledge_bases.py:1435} INFO - ✓ Ingestion job completed successfully
[2025-07-23 03:00:46,075] p95550 {knowledge_bases.py:1438} INFO - Ingestion statistics: {'numberOfDocumentsDeleted': 0, 'numberOfDocumentsFailed': 0, 'numberOfDocumentsScanned': 3, 'numberOfMetadataDocumentsModified': 0, 'numberOfMetadataDocumentsScanned': 0, 'numberOfModifiedDocumentsIndexed': 0, 'numberOfNewDocumentsIndexed': 3}
[2025-07-23 03:00:46,077] p95550 {knowledge_bases.py:1454} INFO - Waiting for final synchronization to complete...


{ 'dataSourceId': 'MMKMP973CF',
  'ingestionJobId': 'YZKQMMB3SV',
  'knowledgeBaseId': 'MIV8LAUJT2',
  'startedAt': datetime.datetime(2025, 7, 23, 8, 0, 35, 932542, tzinfo=tzutc()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 3,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 3},
  'status': 'COMPLETE',
  'updatedAt': datetime.datetime(2025, 7, 23, 8, 0, 43, 128797, tzinfo=tzutc())}


## Creating ReAct Agent

Over the next cells we will create a ReAct agent `restaurant-react` and invoke with the Bedrock Agent's default orchestration and invoke it.

The ReAct approach is an iterative decision-making process where the model analyzes each step, deciding on the next action based on the information gathered at each stage

<center>
  <img src="images/react.png" width="65%" />
</center>

This method provides transparency and allows for a clear, step-by-step breakdown of actions, making it well-suited for workflows that benefit from incremental adjustments. While effective in dynamic environments where real-time re-evaluation is advantageous, ReAct’s sequential structure can introduce latency when high-speed or parallel processing across multiple tools is required.

### Defining agent configuration

Let's now define the configuration for our restaurant assistant. Let's provide some data of what to do in case of situations where the agent cannot answer the user query and some more generic information about the restaurant

```
You are a restaurant assistant helping ‘The Regrettable Experience’ handle reservations. You can talk about the menus, create new bookings, get the details of an existing booking or delete an existing reservation. You reply always politely and mention the name of the restaurant in the reply. NEVER skip the name of the restaurant in the start of a new conversation. If customers ask about anything that you cannot reply, please provide the following phone number for a more personalized experience: +1 999 999 99 9999.

Some information that will be useful to answer your customer's questions:
The Regrettable Experience Address: 101W 87th Street, 100024, New York, New York
Opening hours: 
- Mondays - Fridays: 11am - 2pm and 5pm - 10pm
- Saturdays: 11am - 11pm
- Sundays: 11am - 8pm
```

For this agent, we will also use `Claude 3 Sonnet 3.5 v2` model in order to provide more accurate answers to our users.

For the action group we will provide 3 functions:
* `get_booking_details` to retrieve the details of an existing booking
* `create_booking` to create a new restaurant reservation and
* `delete_booking` to delete an existing reservation

Finally, we also provide some knowledge base configuration including a `kb_instruction` of when to use this knowledge base

In [7]:
agent_name_react = 'restaurant-a-react'
agent_foundation_model = "anthropic.claude-3-5-sonnet-20241022-v2:0"
agent_instruction = """You are a restaurant assistant helping ‘The Regrettable Experience’ handle reservations. 
You can talk about the menus, create new bookings, get the details of an existing booking or delete an existing reservation. 
You reply always politely and mention the name of the restaurant in the reply. 
NEVER skip the name of the restaurant in the start of a new conversation. 
If customers ask about anything that you cannot reply, please provide the following phone number for a more personalized experience: 
+1 999 999 99 9999.

Some information that will be useful to answer your customer's questions:
The Regrettable Experience Address: 101W 87th Street, 100024, New York, New York
Opening hours: 
- Mondays - Fridays: 11am - 2pm and 5pm - 10pm
- Saturdays: 11am - 11pm
- Sundays: 11am - 8pm"""
agent_description = "Agent in charge of a restaurants table bookings"

functions = [
    {
        'name': 'get_booking_details',
        'description': 'Retrieve details of a restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to retrieve",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        'name': 'create_booking',
        'description': 'Create a new restaurant booking',
        'parameters': {
            "date": {
                "description": "The date of the booking in the format YYYY-MM-DD",
                "required": True,
                "type": "string"
            },
            "name": {
                "description": "Name to idenfity your reservation",
                "required": True,
                "type": "string"
            },
            "hour": {
                "description": "The hour of the booking in the format HH:MM",
                "required": True,
                "type": "string"
            },
            "num_guests": {
                "description": "The number of guests for the booking",
                "required": True,
                "type": "integer"
            }
        }
    },
    {
        'name': 'delete_booking',
        'description': 'Delete an existing restaurant booking',
        'parameters': {
            "booking_id": {
                "description": "The ID of the booking to delete",
                "required": True,
                "type": "string"
            }
        }
    },
]

action_group_config_react = {
    'name': 'TableBookingsActionGroup',
    'description': 'Actions for getting table booking information, create a new booking or delete an existing booking',
    'functions': functions,
    'lambda_function_name': f'{agent_name_react}-lambda',
    'lambda_file_path': 'lambda_function.py',
    'environment': {
        'Variables': {
            'booking_table_name': f'{agent_name_react}-table'
        }
    },
    'dynamodb_table_name': f'{agent_name_react}-table',
    'dynamodb_attribute_name': 'booking_id'
}

kb_config = {
    'kb_id': kb_id,
    'kb_instruction': 'Access the knowledge base when customers ask about the plates in the menu.'
}

### Creating agent

In [8]:
ra_react_agent_id, ra_react_agent_alias_id, ra_react_agent_alias_arn, react_orchestration_lambda_function = create_agent(
    agent_name_react,
    agent_instruction,
    agent_foundation_model=agent_foundation_model,
    agent_description=agent_description,
    action_group_config=action_group_config_react,
    kb_config=kb_config,
    create_alias=False
)

creating agent
Policy restaurant-a-react-ba already exists
Checking if AmazonBedrockExecutionRoleForAgents_restaurant-a-react role also exists
Detaching and deleting restaurant-a-react-ba
deleting AmazonBedrockExecutionRoleForAgents_restaurant-a-react
Recreating restaurant-a-react-ba
Detaching AWSLambdaBasicExecutionRole
Detaching AmazonDynamoDBFullAccess
deleting restaurant-a-react-lambda-role
Creating role: restaurant-a-react-lambda-role
Waiting for role to be available...
Attaching basic lambda permissions to restaurant-a-react-lambda-role
Attaching dynamodb permissions to restaurant-a-react-lambda-role
creating and attaching action group
restaurant-a-react-lambda already exists, deleting it and recreating
Table restaurant-a-react-table already exists!
Deleting and recreating it!
Creating table restaurant-a-react-table...
Table restaurant-a-react-table created successfully!
associating knowledge base
Agent id RODVVROXY5 current status: NOT_PREPARED
Waiting for agent status to change

### Getting created agent configuration
Let's now check the agent_id and agent_alias_id values. Those will be required to invoke your agent.
As we did not create an agent version, our agent alias is set to the test value of `TSTALIASID`

In [9]:
ra_react_agent_id, ra_react_agent_alias_id

('RODVVROXY5', 'TSTALIASID')

## Invoking ReAct Agent
Next we will invoke the ReAct agent with a couple of queries. We will use session attributes to pass the current date and customer name in order to make the agent more relatable with a real life restaurant assistant

In [10]:
from datetime import datetime
today = datetime.today().strftime('%b-%d-%Y')
today

'Jul-23-2025'

### Invoking ReAct agent with action group only query
first let's invoke our agent with a query calling only the action group to book a reservation. We will use the magic command `%%time` to measure the latency of our requests

In [11]:
import time
import uuid
session_id:str = str(uuid.uuid1())

query = "Can you make a reservation for 2 people, at 7pm tonight?"
session_state={
    "promptSessionAttributes": { 
         "Customer Name" : "John",
         "Today": today
      },
}
response = invoke_agent_helper(
    query, session_id, ra_react_agent_id, ra_react_agent_alias_id, enable_trace=False, session_state=session_state
)
print(response)

Thank you for choosing The Regrettable Experience! I've successfully made a reservation for you:
- 2 people
- Tonight at 7:00 PM
- Under the name: John
- Booking ID: 4b1f5a6b

We look forward to welcoming you at 101W 87th Street, New York. Please keep your booking ID for future reference.


### Invoking ReAct agent with knowledge base query
Next we will check what is on the menu in order to invoke our agent with a query to the Knowledge Base only

In [12]:
time.sleep(60)

In [13]:
import time
import uuid
session_id:str = str(uuid.uuid1())

query = "What do you serve for dinner?"
response = invoke_agent_helper(
    query, session_id, ra_react_agent_id, ra_react_agent_alias_id, enable_trace=False, session_state=None
)
print(response)

Welcome to The Regrettable Experience! Our dinner menu features a delightful selection of dishes including:

Appetizers:
- Buffalo Chicken Wings with celery sticks and blue cheese dressing

Main Courses:
- Shrimp and Grits: sautéed shrimp in sauce over creamy cheese grits
- Steak with Garlic Butter: grilled sirloin served with loaded mashed potatoes and sautéed vegetables
- Vegetable Stir-Fry with Tofu over jasmine rice in soy-ginger sauce
- Vegetarian Chili topped with cheese and green onions
- Grilled BBQ Chicken with coleslaw and baked beans




### Invoking ReAct agent with Action Group and Knowledge Base query
Now let's try to invoke our agent with a more complex query that requires a plan that will check for the menu and book a reservation

In [14]:
time.sleep(60)

In [15]:
import time
import uuid
session_id:str = str(uuid.uuid1())

query = "What do you serve for dinner? can you make a reservation for 4 people, at 9pm tonight."
session_state={
    "promptSessionAttributes": { 
         "Customer Name" : "Maria",
         "Today": today
      },
}
response = invoke_agent_helper(
    query, session_id, ra_react_agent_id, ra_react_agent_alias_id, enable_trace=True, session_state=session_state
)
print(response)

[2025-07-23 03:05:16,269] p95550 {agents.py:727} INFO - None
[2025-07-23 03:05:16,417] p95550 {agents.py:750} INFO - {
  "agentAliasId": "TSTALIASID",
  "agentId": "RODVVROXY5",
  "agentVersion": "DRAFT",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-west-2:071040227595:agent-alias/RODVVROXY5/TSTALIASID"
    }
  ],
  "eventTime": "2025-07-23T08:05:16.428023+00:00",
  "sessionId": "c44d4c5c-679b-11f0-bc83-8a268b0e34f8",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "foundationModel": "anthropic.claude-3-5-sonnet-20241022-v2:0",
        "inferenceConfiguration": {
          "maximumLength": 2048,
          "stopSequences": [
            "</invoke>",
            "</answer>",
            "</error>"
          ],
          "temperature": 0.0,
          "topK": 250,
          "topP": 1.0
        },
        "text": "{\"system\":\" You are a restaurant assistant helping \\u2018The Regrettable Experience\\u2019 handle reservations.  You ca

{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Wed, 23 Jul 2025 08:05:16 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': 'c44d4c5c-679b-11f0-bc83-8a268b0e34f8',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': '39032d27-81e9-4a72-b5d1-22a573c0f007'},
                      'HTTPStatusCode': 200,
                      'RequestId': '39032d27-81e9-4a72-b5d1-22a573c0f007',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x120d2c1d0>,
 'contentType': 'application/json',
 'sessionId': 'c44d4c5c-679b-11f0-bc83-8a268b0e34f8'}


[2025-07-23 03:05:22,577] p95550 {agents.py:750} INFO - {
  "agentAliasId": "TSTALIASID",
  "agentId": "RODVVROXY5",
  "agentVersion": "DRAFT",
  "callerChain": [
    {
      "agentAliasArn": "arn:aws:bedrock:us-west-2:071040227595:agent-alias/RODVVROXY5/TSTALIASID"
    }
  ],
  "eventTime": "2025-07-23T08:05:22.583706+00:00",
  "sessionId": "c44d4c5c-679b-11f0-bc83-8a268b0e34f8",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationOutput": {
        "metadata": {
          "clientRequestId": "9336f885-610c-41a5-97d0-b3eaf62771c8",
          "endTime": "2025-07-23T08:05:22.582354+00:00",
          "startTime": "2025-07-23T08:05:16.428393+00:00",
          "totalTimeMs": 6154,
          "usage": {
            "inputTokens": 1648,
            "outputTokens": 156
          }
        },
        "rawResponse": {
          "content": "{\"stop_sequence\":null,\"type\":\"message\",\"id\":\"msg_bdrk_01MmfNfPcnBaYSX5okiAUW4h\",\"content\":[{\"imageSource\":null,\"reasoningTextSignatur

Welcome to The Regrettable Experience! 

For dinner, we serve delicious options including Buffalo Chicken Wings with celery sticks and blue cheese dressing, as well as our signature Shrimp and Grits featuring sautéed shrimp over creamy cheese grits.

I've successfully made a reservation for you:
- Date: Tonight (July 23, 2025)
- Time: 9:00 PM
- Number of guests: 4
- Booking ID: 042fa5dd

Please keep your booking ID for reference. We look forward to serving you tonight!




## Creating ReWoo Agent
Over the next cells we will create a ReWoo agent `restaurant-rewoo` using Bedrock Agent's custom orchestrator and we will invoke it with the the same customer orchestrator.

The ReWOO technique optimizes performance by generating a complete task plan up front and executing it without checking intermediate outputs.

<center>
  <img src="images/rewoo.png" width="85%" />
</center>

This approach minimizes model calls, potentially reducing response times. For tasks where speed is prioritized over iterative adjustments—or where the intermediate reasoning steps should remain hidden for security reasons—ReWOO offers clear advantages over the default ReAct strategy.

### Defining agent configuration
The agent configuration remains basicaly the same, with the exception of the custom orchestration lambda that needs to be created. The file `lambda_rewoo.py` has the code for the orchestration. 

The custom orchestrator enables dynamic decision-making and adaptable workflow management through contract-based interactions between Amazon Bedrock Agents and AWS Lambda. The AWS Lambda function acts as the orchestration engine, processing contextual inputs—such as state, conversation history, session parameters, and user requests—to generate instructions and define the state for subsequent actions. Upon receiving user input, Amazon Bedrock Agents uses the custom orchestrator logic and the [Amazon Bedrock Converse API](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_Converse.html) to manage interactions between the underlying foundation model and various tools, such as action groups, knowledge bases, and guardrails.


The following diagram illustrates the flow of interactions between the user, Amazon Bedrock Agents, and the custom orchestrator, which manages the workflow:
![custom orchestrator](images/custom_orchestrator.png)

In [16]:
agent_name_rewoo = 'restaurant-a-rewoo'
custom_orchestration_lambda_name = 'rewoo-o-lambda'
custom_orchestration_lambda_rewoo = {
    'lambda_function_name': custom_orchestration_lambda_name,
    'lambda_file_path': 'lambda_rewoo.py'
}
action_group_config_rewoo = {
    'name': 'TableBookingsActionGroup',
    'description': 'Actions for getting table booking information, create a new booking or delete an existing booking',
    'functions': functions,
    'lambda_function_name': f'{agent_name_rewoo}-lambda',
    'lambda_file_path': 'lambda_function.py',
    'environment': {
        'Variables': {
            'booking_table_name': f'{agent_name_rewoo}-table'
        }
    },
    'dynamodb_table_name': f'{agent_name_rewoo}-table',
    'dynamodb_attribute_name': 'booking_id'
}
ra_rewoo_agent_id, ra_rewoo_agent_alias_id, ra_rewoo_agent_alias_arn, rewoo_orchestration_lambda_function = create_agent(
    agent_name_rewoo,
    agent_instruction,
    agent_foundation_model=agent_foundation_model,
    agent_description=agent_description,
    action_group_config=action_group_config_rewoo,
    kb_config=kb_config,
    custom_orchestration_lambda=custom_orchestration_lambda_rewoo,
    create_alias=False
)

creating agent
Creating role: restaurant-a-rewoo-lambda-role
Waiting for role to be available...
Attaching basic lambda permissions to restaurant-a-rewoo-lambda-role
Attaching dynamodb permissions to restaurant-a-rewoo-lambda-role
creating and attaching action group
Creating table restaurant-a-rewoo-table...
Table restaurant-a-rewoo-table created successfully!
associating knowledge base
Agent id MNTRB27OYX current status: NOT_PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id MNTRB27OYX current status: PREPARED


### Getting created agent configuration
Let's now check the agent_id and agent_alias_id values. Those will be required to invoke your agent.
As we did not create an agent version, our agent alias is set to the test value of `TSTALIASID`

In [17]:
ra_rewoo_agent_id, ra_rewoo_agent_alias_id

('MNTRB27OYX', 'TSTALIASID')

## Invoking ReWoo Agent
Next we will invoke the ReWoo agent with a couple of queries. We will still use session attributes to pass the current date and customer name in order to make the agent more relatable with a real life restaurant assistant.

To use custom orchestrator, you need to pass the orchestration lambda ARN via `sessionAttribute` in the `sessionState` parameter

### Invoking ReWoo agent with action group only query
first let's invoke our agent with a query calling only the action group to book a reservation. We will use the magic command `%%time` to measure the latency of our requests

In [18]:
time.sleep(60)

In [19]:
import time
import uuid
session_id:str = str(uuid.uuid1())
session_state={
    "promptSessionAttributes": { 
         "Customer Name" : "John",
         "Today": today
      },
    'sessionAttributes': {
        'lambda': rewoo_orchestration_lambda_function['FunctionArn']
    }
}
query = "Can you make a reservation for 2 people, at 7pm tonight?"
response = invoke_agent_helper(
    query, session_id, ra_rewoo_agent_id, ra_rewoo_agent_alias_id, enable_trace=False, session_state=session_state
)
print(response)

"I've made a reservation for 2 people tonight at 7:00 PM. Your booking ID is 205a12ee. Please keep this booking ID for future reference in case you need to modify or cancel your reservation.\n\nIs there anything else I can help you with?"


### Invoking ReWoo agent with knowledge base query
Next we will check what is on the menu in order to invoke our agent with a query to the Knowledge Base only

In [ ]:
time.sleep(60)

In [20]:
import time
import uuid
session_id:str = str(uuid.uuid1())
session_state={
    'sessionAttributes': {
        'lambda': rewoo_orchestration_lambda_function['FunctionArn']
    }
}
query = "What do you serve for dinner?"
response = invoke_agent_helper(
    query, session_id, ra_rewoo_agent_id, ra_rewoo_agent_alias_id, enable_trace=False, session_state=session_state
)
print(response)

"Based on the menu information available, for dinner we serve:\n\n1. Buffalo Chicken Wings\n- Served with celery sticks and blue cheese dressing\n- Contains allergens: Dairy (in blue cheese dressing), Gluten (in coating), possible Soy (in sauce)\n\n2. Shrimp and Grits\n- Features succulent shrimp saut\u00e9ed in a flavorful sauce, served over creamy cheese grits\n- Contains allergens: Dairy (in cheese grits)\n\nLet me search for any additional dinner entr\u00e9es."


### Invoking ReWoo agent with Action Group and Knowledge Base query
Now let's try to invoke our agent with a more complex query that requires a plan that will check for the menu and book a reservation

In [ ]:
time.sleep(60)

In [21]:
import time
import uuid
session_id:str = str(uuid.uuid1())
query = "What do you serve for dinner? can you make a reservation for 4 people, at 9pm tonight."
session_state={
    "promptSessionAttributes": { 
         "Customer Name" : "John",
         "Today": today
      },
    'sessionAttributes': {
        'lambda': rewoo_orchestration_lambda_function['FunctionArn']
    }
}
response = invoke_agent_helper(
    query, session_id, ra_rewoo_agent_id, ra_rewoo_agent_alias_id, enable_trace=True, session_state=session_state
)
print(response)

[2025-07-23 03:15:16,553] p95550 {agents.py:727} INFO - None


{'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-type': 'application/vnd.amazon.eventstream',
                                      'date': 'Wed, 23 Jul 2025 08:15:16 GMT',
                                      'transfer-encoding': 'chunked',
                                      'x-amz-bedrock-agent-session-id': '2a1fa11e-679d-11f0-bc83-8a268b0e34f8',
                                      'x-amzn-bedrock-agent-content-type': 'application/json',
                                      'x-amzn-requestid': 'f57baf2a-2f6f-4cd9-8c3a-68a5e8ae926a'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'f57baf2a-2f6f-4cd9-8c3a-68a5e8ae926a',
                      'RetryAttempts': 0},
 'completion': <botocore.eventstream.EventStream object at 0x120d46930>,
 'contentType': 'application/json',
 'sessionId': '2a1fa11e-679d-11f0-bc83-8a268b0e34f8'}


[2025-07-23 03:15:16,754] p95550 {agents.py:750} INFO - {
  "agentAliasId": "TSTALIASID",
  "agentId": "MNTRB27OYX",
  "agentVersion": "DRAFT",
  "sessionId": "2a1fa11e-679d-11f0-bc83-8a268b0e34f8",
  "trace": {
    "customOrchestrationTrace": {
      "event": {
        "text": "This is on start debug trace!"
      },
      "traceId": "f57baf2a-2f6f-4cd9-8c3a-68a5e8ae926a-0"
    }
  }
}
[2025-07-23 03:15:16,757] p95550 {agents.py:750} INFO - {
  "agentAliasId": "TSTALIASID",
  "agentId": "MNTRB27OYX",
  "agentVersion": "DRAFT",
  "sessionId": "2a1fa11e-679d-11f0-bc83-8a268b0e34f8",
  "trace": {
    "orchestrationTrace": {
      "modelInvocationInput": {
        "foundationModel": "anthropic.claude-3-5-sonnet-20241022-v2:0",
        "inferenceConfiguration": {
          "maximumLength": 500,
          "stopSequences": [],
          "temperature": 0.0,
          "topP": 0.8999999761581421
        },
        "text": "{\"messages\":[{\"role\":\"user\",\"content\":[{\"text\":\"\\nYou are a 

"From our dinner menu, we serve several items including:\n1. Buffalo Chicken Wings - served with celery sticks and blue cheese dressing\n2. Shrimp and Grits - succulent shrimp saut\u00e9ed in a flavorful sauce, served over creamy cheese grits\n\nI've made a reservation for 4 people at 9:00 PM tonight. Your booking ID is bcad3a54. Please keep this ID for your records.\n\nNote: I used a placeholder name \"John\" for the reservation. Would you like me to update the reservation with a different name?"


## Comparing ReAct and ReWoo orchestrations

As we can see in the invocations before, the latency to run simple queries in ReAct and ReWoo is similar. However, with complex multi-step queries, the latency to run a ReWoo orchestration is significantly lower. 

The videos below show case the processing steps to process the query

```
What do you serve for dinner? can you make a reservation for 4 people, at 9pm tonight.
```

For this query, **ReAct** will:
- create a plan to solve the task 1st checking what is served in the dinner menu and then book a reservation
- check the knowledge base for what is served in dinner menu
- evaluate if the plan is still proper to solve the task 
- book the reservation
- evaluate if the plan is still proper to solve the task
- create a final response with the dinner options and booking reservation

In [22]:
from IPython.display import HTML

HTML("""
<video alt="test" controls width="90%">
    <source src="images/react_flow.mp4" type="video/mp4">
</video>
""")

For the same query, **ReWoo** will:
- create a plan to solve the task checking what is served in the dinner menu and booking a reservation
- check the knowledege base
- book a reservation
- create a final response with the dinner options and booking reservation

In [23]:
from IPython.display import HTML

HTML("""
<video alt="test" controls width="100%">
    <source src="images/rewoo_flow.mp4" type="video/mp4">
</video>
""")

## [Optional] Clean up

In this optional step we will delete the created resources to avoid unecessary costs

In [ ]:
# # clean up react agent
# clean_up_resources(
#     agent_name_react,
#     custom_orchestration_lambda_function_name=None,
#     dynamodb_table=f'{agent_name_react}-table'
# )

In [ ]:
# # clean up rewoo agent
# clean_up_resources(
#     agent_name_rewoo,
#     custom_orchestration_lambda_function_name=custom_orchestration_lambda_name,
#     dynamodb_table=f'{agent_name_rewoo}-table'
# )

In [ ]:
# # delete kb
# kb.delete_kb(
#     kb_name=knowledge_base_name, delete_s3_bucket=True, delete_iam_roles_and_policies=True
# )

## Next Steps

Congratulations, you have created your first custom orchestrator agent!

As next steps we suggest you experiment with other orchestration strategies. This folder also provides some starting examples for ReAct and ReWoo orchestration using JavaScript and Python code:
- `custom_orchestrators_samples/lambda_react.js` file contains a JavaScript implementation of ReAct. 
- `custom_orchestrators_samples/lambda_react.py` file contains a Python implementation of ReAct. 
- `custom_orchestrators_samples/lambda_rewoo.js` file contains a JavaScript implementation of ReWoo. 

You can use these file to change the default behavior of Bedrock Agent's ReAct implementation and to start creating your own orchestration code. 

**Disclaimer:** Those code samples are provided as a start point for your application. You should validate and update them accordingly to your use case